# Final project : Last part

For the last part of the project, the idea is to create a wine recommender that can, based on the taste of the person, propose a new wine to try.

# Objective

There are several objectives to make a wine recommender :

- Help the sommelier to discover new wines that he wouldn't think could be great for example
- Help the beginners to choose between the thousands bottles of wine based on their taste

By using this wine recommender, it will, in a professional way (the sommelier for example), save a lot of time to find other wonderful wines so reduce the costs as well.
In a non-professional way, it will definitely help the beginners to choose and save time as well.

# Importing the data

In [1]:
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np

In [2]:
df_r = pd.read_excel('/Users/accountmanager/Desktop/Iron Hack/Final project/winequality-red.xls')
df_w = pd.read_excel('/Users/accountmanager/Desktop/Iron Hack/Final project/winequality-white.xls')

# Starting with white wine

In [3]:
# Considering the fact that we dont have the names because of privacy, I will add an ID to every wine in order to -
# - identify them easily

In [4]:
df_w.insert(0, 'id', range(1, len(df_w) + 1))
df_w

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,1,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,2,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,3,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,5,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,4894,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,4895,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,4896,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,4897,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [5]:
# Deleting the quality since it is not a relevant feature in clustering

df_w = df_w.drop('quality', axis = 1)

In [6]:
X = df_w.drop('id', axis = 1)
# y = df['artist']

scaler = StandardScaler()

X_prep = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X_prep)

clusters = kmeans.predict(X_prep)

scaled_df = pd.DataFrame(X_prep, columns=X.columns)
scaled_df['id'] = df_w['id']
scaled_df['cluster'] = clusters
scaled_df = scaled_df.round(3)
scaled_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,id,cluster
0,0.172,-0.082,0.213,2.821,-0.035,0.570,0.745,2.332,-1.247,-0.349,-1.393,1,1
1,-0.658,0.216,0.048,-0.945,0.148,-1.253,-0.150,-0.009,0.740,0.001,-0.824,2,0
2,1.476,0.017,0.544,0.100,0.194,-0.312,-0.973,0.359,0.475,-0.437,-0.337,3,2
3,0.409,-0.479,-0.117,0.416,0.560,0.688,1.121,0.526,0.011,-0.787,-0.499,4,1
4,0.409,-0.479,-0.117,0.416,0.560,0.688,1.121,0.526,0.011,-0.787,-0.499,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,-0.776,-0.677,-0.365,-0.945,-0.310,-0.665,-1.091,-0.965,0.541,0.089,0.557,4894,0
4894,-0.302,0.414,0.213,0.317,0.056,1.276,0.697,0.292,-0.253,-0.262,-0.743,4895,1
4895,-0.420,-0.379,-1.192,-1.024,-0.218,-0.312,-0.644,-0.497,-1.313,-0.262,-0.906,4896,2
4896,-1.606,0.117,-0.283,-1.043,-1.088,-0.900,-0.667,-1.785,1.005,-0.963,1.858,4897,0


In [7]:
scaled_df.groupby(['cluster', 'alcohol'], as_index=False).count().sort_values(['cluster', 'id'], ascending=[True, False])[['alcohol', 'cluster', 'id']].reset_index(drop=True)



,alcohol,cluster,id
0,0.395,0,83
1,-0.012,0,68
2,-0.418,0,59
3,0.232,0,58
4,-0.093,0,56
...,...,...,...
189,0.747,2,1
190,0.774,2,1
191,1.086,2,1
192,1.411,2,1


In [8]:
wine_id = int(input('Insert the id of your choice: '))
results = df_w.loc[df_w['id'] == wine_id]

df_ = pd.DataFrame(results)
new_features = df_[X.columns]

scaled_x = scaler.transform(new_features)
cluster = kmeans.predict(scaled_x)

filtered_df = np.array(scaled_df[scaled_df['cluster'] == cluster[0]][X.columns], order="C")
closest, _ = pairwise_distances_argmin_min(scaled_x, filtered_df)

Insert the id of your choice: 34


In [9]:
scaled_df.loc[closest[0]]['alcohol'], scaled_df.loc[closest[0]]['id']

(0.232, 13.0)

In [10]:
def wine_recommender():
    wine_id = int(input('Insert the id of your choice : '))
    results = df_w.loc[df_w['id'] == wine_id]
    
    df_ = pd.DataFrame(results)
    new_features = df_[X.columns]

    scaled_x = scaler.transform(new_features)
    cluster = kmeans.predict(scaled_x)

    filtered_df = np.array(scaled_df[scaled_df['cluster'] == cluster[0]][X.columns], order="C")
    closest, _ = pairwise_distances_argmin_min(scaled_x, filtered_df)
    
    print('Considering your taste, you should try this wine which is similar to the one you put :')
    return scaled_df.loc[closest]['id'].values[0]

In [11]:
wine_recommender()

Insert the id of your choice : 56
Considering your taste, you should try this wine which is similar to the one you put :


17

In [12]:
results = df_w.loc[df_w['id'] == 1109]
    
dftest = pd.DataFrame(results)
dftest

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1108,1109,6.1,0.29,0.27,1.7,0.024,13.0,76.0,0.9893,3.21,0.51,12.6


In [13]:
results = df_w.loc[df_w['id'] == 3540]
    
dftest = pd.DataFrame(results)
dftest

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
3539,3540,7.2,0.27,0.31,1.2,0.031,27.0,80.0,0.98892,3.03,0.33,12.7


In [14]:
df_w['fixed acidity'].describe()

count    4898.000000
mean        6.854788
std         0.843868
min         3.800000
25%         6.300000
50%         6.800000
75%         7.300000
max        14.200000
Name: fixed acidity, dtype: float64

In [15]:
df_w['residual sugar'].describe()

count    4898.000000
mean        6.391415
std         5.072058
min         0.600000
25%         1.700000
50%         5.200000
75%         9.900000
max        65.800000
Name: residual sugar, dtype: float64

In [16]:
df_w['alcohol'].describe()

count    4898.000000
mean       10.514267
std         1.230621
min         8.000000
25%         9.500000
50%        10.400000
75%        11.400000
max        14.200000
Name: alcohol, dtype: float64

In [17]:
# Improving the function by adding some explanations on the wine in question (using relevant indicators).

def wine_recommender2():
    wine_id = int(input('Insert a wine of your choice using the id : '))
    results = df_w.loc[df_w['id'] == wine_id]
    
    df_ = pd.DataFrame(results)
    new_features = df_[X.columns]

    scaled_x = scaler.transform(new_features)
    cluster = kmeans.predict(scaled_x)

    filtered_df = np.array(scaled_df[scaled_df['cluster'] == cluster[0]][X.columns], order="C")
    closest, _ = pairwise_distances_argmin_min(scaled_x, filtered_df)
    
    # Adding variables here (there might be a faster way for sure)
    
    fixed_acidity33 = df_w['fixed acidity'].quantile(0.33)
    residual_sugar33 = df_w['residual sugar'].quantile(0.33)
    alcohol33 = df_w['alcohol'].quantile(0.33)
    
    fixed_acidity66 = df_w['fixed acidity'].quantile(0.66)
    residual_sugar66 = df_w['residual sugar'].quantile(0.66)
    alcohol66 = df_w['alcohol'].quantile(0.66)
    
    print('you choose a wine that has the following caracterisitics :')
        

    if df_w['fixed acidity'].loc[df_w['id'] == wine_id].item() < fixed_acidity33:
        print('- Low acidity')
    elif df_w['fixed acidity'].loc[df_w['id'] == wine_id].item() < fixed_acidity66:
        print('- Normal acidity')
    else :
        print('- High acidity')
    
    
    if df_w['residual sugar'].loc[df_w['id'] == wine_id].item() < residual_sugar33:
        print('- Low in sugar')
    elif df_w['residual sugar'].loc[df_w['id'] == wine_id].item() < fixed_acidity66:
        print('- Average quantity of sugar')
    else :
        print('- High quantity of sugar')
        
    
    if df_w['alcohol'].loc[df_w['id'] == wine_id].item() < alcohol33:
        print('- Low alcoholic degree')
    elif df_w['alcohol'].loc[df_w['id'] == wine_id].item() < fixed_acidity66:
        print('- Normal alcoholic degree')
    else :
        print('- High alcoholic degree')
    
    
    
    
    final_res = print('Considering your taste, you should try this wine which is similar to the one you put :', scaled_df.loc[closest]['id'].values[0])
    return final_res

In [18]:
wine_recommender2()

Insert a wine of your choice using the id : 54
you choose a wine that has the following caracterisitics :
- Low acidity
- Low in sugar
- High alcoholic degree
Considering your taste, you should try this wine which is similar to the one you put : 20


Just some test after

In [20]:
df_w['fixed acidity'].loc[df_w['id'] == wine_id].item()

6.2

In [21]:
wine_id = 58

fixed_acidity33 = df_w['fixed acidity'].quantile(0.33)
residual_sugar33 = df_w['residual sugar'].quantile(0.33)
alcohol33 = df_w['alcohol'].quantile(0.33)
    
fixed_acidity66 = df_w['fixed acidity'].quantile(0.33)
residual_sugar66 = df_w['residual sugar'].quantile(0.33)
alcohol_66 = df_w['alcohol'].quantile(0.33)

if df_w['alcohol'].loc[df_w['id'] == wine_id].item() < fixed_acidity33:
        print('- Low alcoholic degree')
elif df_w['alcohol'].loc[df_w['id'] == wine_id].item() < fixed_acidity66:
        print('- Normal alcoholic degree')
else :
        print('- High alcoholic degree')

- High alcoholic degree


In [22]:
df_w['alcohol'].loc[df_w['id'] == wine_id].item()

8.9

In [23]:
print('ahahah ', 1)

ahahah  1


In [26]:
df_w[df_w['id']==2300]

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
2299,2300,7.0,0.32,0.31,6.4,0.031,38.0,115.0,0.99235,3.38,0.58,12.2


In [27]:
df_w[df_w['id']==640]

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
639,640,5.8,0.335,0.14,5.8,0.046,49.0,197.0,0.9937,3.3,0.71,10.3
